In [28]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
!tesseract --version

'tesseract' is not recognized as an internal or external command,
operable program or batch file.


In [29]:
import cv2
import numpy as np
import pytesseract
import pandas as pd
import re
import os

# --- KONFIGURASI PENTING ---
# Untuk Windows, uncomment baris ini dan sesuaikan path tesseract:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 1. TENTUKAN NAMA FOLDER
folder_path = r"C:\Users\LENOVO\Downloads\statexplore\data\c10\consumption_pct_gdp"

# 2. DAFTAR NAMA FILE GAMBAR
image_files = sorted([
    f for f in os.listdir(folder_path) if f.endswith(".png")
])

print(f"Memulai proses ekstraksi dari folder: '{folder_path}'")
print("=" * 50)

# List gabungan semua data
all_points = []

# Loop melalui setiap file
for filename in image_files:
    full_path = os.path.join(folder_path, filename)

    try:
        img = cv2.imread(full_path)
        if img is None:
            print(f"[ERROR] Tidak dapat membaca gambar: {full_path}\n" + "-"*40)
            continue

        h, w, _ = img.shape

        # Coba deteksi tahun pakai OCR
        title_area = img[0:int(h*0.1), 0:w]
        title_text = pytesseract.image_to_string(title_area)
        match = re.search(r'\b(20\d{2})\b', title_text)

        if match:
            year = match.group(1)
        else:
            # fallback ambil dari nama file
            year_match = re.search(r'(\d{4})', filename)
            year = year_match.group(1) if year_match else "UNKNOWN"

        print(f"[INFO] Memproses file: {filename} -> Tahun terdeteksi: {year}")

        # Area plot
        plot_area = img[int(h*0.12):int(h*0.9), int(w*0.05):int(w*0.98)]
        plot_h, plot_w, _ = plot_area.shape

        # Konversi ke HSV & buat mask biru
        hsv = cv2.cvtColor(plot_area, cv2.COLOR_BGR2HSV)
        lower_blue = np.array([100, 150, 50])
        upper_blue = np.array([140, 255, 255])
        mask = cv2.inRange(hsv, lower_blue, upper_blue)

        extracted_points = []
        Y_MIN_VALUE, Y_MAX_VALUE = 52, 72

        for x_pixel in range(plot_w):
            y_pixels = np.where(mask[:, x_pixel] > 0)[0]
            if len(y_pixels) > 0:
                y_pixel_avg = int(np.mean(y_pixels))
                value = Y_MAX_VALUE - ((y_pixel_avg / plot_h) * (Y_MAX_VALUE - Y_MIN_VALUE))
                day_of_year = int((x_pixel / plot_w) * 365) + 1
                extracted_points.append({
                    'Year': int(year),
                    'DayOfYear': day_of_year,
                    'Value': round(value, 2)
                })

        if not extracted_points:
            print(f"[WARNING] Tidak ada titik data di {filename}\n" + "-"*40)
            continue

        # Tambahkan ke list global
        all_points.extend(extracted_points)

        print(f"[SUCCESS] Data dari {filename} berhasil diekstrak")
        print("-" * 40)

    except Exception as e:
        print(f"[ERROR] Gagal proses {filename}: {e}\n" + "-"*40)

# Setelah semua file diproses, gabungkan ke 1 CSV
if all_points:
    df_all = pd.DataFrame(all_points).drop_duplicates(subset=['Year','DayOfYear'], keep='first')
    df_all.to_csv("consumption_pct_gdp_all.csv", index=False)
    print("✅ Semua data berhasil digabung ke consumption_pct_gdp_all.csv")
else:
    print("⚠️ Tidak ada data yang berhasil diekstrak.")

Memulai proses ekstraksi dari folder: 'C:\Users\LENOVO\Downloads\statexplore\data\c10\consumption_pct_gdp'
[INFO] Memproses file: consumption_pct_gdp tahun 2000.png -> Tahun terdeteksi: 2000
[SUCCESS] Data dari consumption_pct_gdp tahun 2000.png berhasil diekstrak
----------------------------------------
[INFO] Memproses file: consumption_pct_gdp tahun 2001.png -> Tahun terdeteksi: 2001
[SUCCESS] Data dari consumption_pct_gdp tahun 2001.png berhasil diekstrak
----------------------------------------
[INFO] Memproses file: consumption_pct_gdp tahun 2002.png -> Tahun terdeteksi: 2002
[SUCCESS] Data dari consumption_pct_gdp tahun 2002.png berhasil diekstrak
----------------------------------------
[INFO] Memproses file: consumption_pct_gdp tahun 2003.png -> Tahun terdeteksi: 2003
[SUCCESS] Data dari consumption_pct_gdp tahun 2003.png berhasil diekstrak
----------------------------------------
[INFO] Memproses file: consumption_pct_gdp tahun 2004.png -> Tahun terdeteksi: 2004
[SUCCESS] Dat